##setup

install ananke (note I am using the *stable* branch not the *main* branch---we are in JOSS review and the main branch is frozen till that is finished). the stable branch has some very useful upgrades.

import a couple packages

In [ ]:
import numpy as np
import ananke as an
an.__version__

##example 0: Gaussian "galaxy" blob

We define here some dummy input data. Py-ananke has a method to produce such data.

In [ ]:
np.random.seed(0)
p = an.Ananke.make_dummy_particles_input()
p.keys()

The input data must be formatted as a dictionary of equal-length arrays. The dictionary must have the following entries:
- key `pos3`: particle position coordinates in $kpc$ (shape Nx3)
- key `vel3`: particle velocity coordinates in $km.s^{-1}$ (shape Nx3)
- key `mass`: particle stellar mass in solar masses
- key `age`: particle log10 stellar age in years
- key `feh`: particle stellar metallicity \[Fe/H\] in dex relative to solar

Additionally, the following entries can optionally be added:
- key `parentid`: index to give to the parent particle
- key `id`: additional index to classify the parent particle
- key `log10_NH`: log10 hydrogen column densities between Observer position and particle in $cm^{-2}$ - must be provided to estimate extinctions
- key `dform`: particle formation distance
- keys `helium`, `carbon`, `nitrogen`, `oxygen`, `neon`, `magnesium`, `silicon`, `sulphur`, `calcium`: particle various chemical abundances \[X/H\]
- key `alpha`: particle alpha chemical abundances \[Mg/Fe\]

Ananke will compute the phase space densities that are used to determine particle smoothing lengths, but one can include pre-computed densities with the following entries:
- key `rho_pos`: particle density in position space in $kpc^{-3}$
- key `rho_vel`: particle density in velocity space in $km^{-3}.s^{3}$

We can define here some parameters for Ananke such as
 - the observer position `observer`
 - the shell of particles to mask `rshell`
 - the sampling factor `fsample`
 - the photometric system of choise `photo_sys`
 - the CMD `cmd_magnames` and its box limits `cmd_box`

for the photometric system, you can choose from the list generated by this command:

In [ ]:
an.display_available_photometric_systems()

You can see what the different photometric bands are called by querying the selected system:

In [ ]:
ps = an.display_available_photometric_systems()['padova']['GAIA__DR2']
ps.mag_names

With this in mind we choose an observer at a random location 50 kpc from the center of the MW -- this is like if we lived in the LMC =)

In [ ]:
D = 50 # *units.kpc

#put the observer along a random vector
observer = np.nan*np.ones(3)
while not np.linalg.norm(observer)<1:
    observer = 2*np.random.rand(3)-1

observer *= D/np.linalg.norm(observer)

#select the range in galactocentric radius for which you want to generate particles
rshell = [0, 2*D]

#subsample fraction -- useful for testing before generating a full survey, and for estimating final data volume
fsample = 0.01

#which instrument is surveying ?
photo_sys = 'padova/GAIA__DR2'


#how are you describing the magnitude/color limits of the survey?
#here we're using Gaia G magnitude and BP-RP color
cmd_magnames = {'magnitude': 'G',
                'color_minuend': 'Gbp',
                'color_subtrahend': 'Grp'}

#you can place optional limits in either absolute or apparent magnitude or color
cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

For more details regarding these parameters and more, you may consult the docstring associated to the class `Ananke`:

In [ ]:
help(an.Ananke)

Initialising the survey with the defined quantities above:

In [ ]:
name = 'blob'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

The method `run` runs the pipeline:

In [ ]:
survey = ananke.run()

The output is saved as a `vaex` dataframe, with its columns organized in alphabetical order. These notably include:
- key `A_0` for the reference extinction which extinction coefficients are based on (at $\lambda = 550 \, nm$ in the case of Gaia DR2)
- key `A_{filter_name}` for the extinction value in each filter designated by `filter_name` (in this case, where `filter_name` is one of the 3 Gaia bands `gaia_g_bpmag`, `gaia_g_rpmag` \& `gaia_gmag`)
- key `E(B-V)` for the reddening index
- key `age` for the log10 stellar age in years
- key `alpha`, `calcium`, `carbon`, `helium`, `magnesium`, `neon`, `nitrogen`, `oxygen`, `silicon`, `sulphur` for the various chemical abundances as given as input
- key `dec`, `ra` for the astrometric declination and right ascension celestial coordinates in degrees
- key `dform` for the formation distance as given as input
- key `dmod` for the distance modulus
- key `feh` for the stellar metallicity \[Fe/H\] in dex relative to solar
- key `glat`, `glon` for the astrometric galactic latitude and longitude celestrial coordinates in degrees
- key `grav` for the log10 surface gravity in CGS units
- key `log10_NH` for the log10 hydrogen column density between Observer position and star in $cm^{-2}$
- key `lum` for the stellar luminosity in solar luminosities
- key `mact`, `mtip`, `mini` for respectively the current stellar mass, the mass of that same star at tip of giant branch for its given age \& metallicity and its stellar mass on zero-age main sequence, all in solar masses
- key `mub`, `mudec`, `mul`, `mura` for the astrometric proper motions, respectively in the direction of the galactic latitude, declination, galactic longitude and right ascension, all in milliarcseconds per year
- key `parentid` for the parent particle index as given as input
- key `partid` for the flag that identifies stars that are *not* central relatively to their parent particle
- key `pi` for the star parallax in milliarcseconds
- key `px`, `py`, `pz` for the star position cartesian coordinates in $kpc$ relative to the Observer's position
- key `rad` for the star distance to the Observer in $kpc$
- key `teff` for the star effective temperature in Kelvin
- key `vr` for the star astrometric radial velocity in $km.s^{-1}$
- key `vx`, `vy`, `vz` for the star velocity cartesian coordinates in $km.s^{-1}$ relative to the Observer's velocity

Additionally, astrometric and photometric quantities `X` all have associated columns identified as:
- key `X_Sig` for the standard error on the quantity `X`
- key `X_Err` for the actual drawn gaussian error on the quantity `X`

In [ ]:
survey

Please refer to [`vaex`'s documentation](https://vaex.io/docs/tutorial.html) for further help on how to use `vaex` dataframes: the following line for example isolate only the rows with non-NaN photometry.

In [ ]:
survey[~survey.gaia__dr2_g.isna()]

In [ ]:
from matplotlib import pyplot as pl
from matplotlib.colors import LogNorm

select only stars that are not extincted (have an observed magnitude that is not NaN)

In [ ]:
survey_observed = survey[~survey.gaia__dr2_g.isna()]


CMD as "observed" by our survey

In [ ]:
pl.hist2d(survey_observed.gaia__dr2_gbp.values - survey_observed.gaia__dr2_grp.values, survey_observed.gaia__dr2_g.values, bins=100, norm=LogNorm())
pl.ylim(21.3,15)

the "true" CMD -- corrected for distance and extinction/reddening

In [ ]:
pl.hist2d(survey_observed.gaia__dr2_gbp_Intrinsic.values - survey_observed.gaia__dr2_grp_Intrinsic.values, survey_observed.gaia__dr2_g_Intrinsic.values, bins=100, norm=LogNorm())
pl.ylim(10,-4)

In [ ]:
from astropy import coordinates as apc
from astropy import units as u

The code below "wraps" the stars' ra and dec so the main "bulge" is not split across 0<->2π (remember we put ourselves at some random angle so the galaxy center will not be at galactic longitude of zero necessarily)

In [ ]:
ra = apc.Angle(survey_observed.ra.values*u.deg).wrap_at(180*u.deg)
dec = apc.Angle(survey_observed.dec.values*u.deg).wrap_at(180*u.deg)

In [ ]:
pl.hist2d(ra.value, dec.value, bins=[200,100], norm=LogNorm());

pl.xlim(-180,180)
pl.ylim(-90,90)

##example 1: a stream

In the previous example the galaxy was just a Gaussian blob. Let's do a more complicated example---a stream from the FIRE simulations (see Panithanpaisal et al. 2021)

In [ ]:
sample = np.load('example_stream.npz')

The file includes all the things needed to resample, plus a `parentid` to reconnect to the parent simulation that will be passed along

In [ ]:
sample.files

In [ ]:
p={}
for f in sample.files:
    p[f]=sample[f]

In [ ]:
pl.plot(p['pos3'][:,0],p['pos3'][:,2], '*m')
pl.plot(observer[0],observer[2],'*y',ms=15,mec='k')

In [ ]:
pl.plot(p['pos3'][:,0],p['pos3'][:,1], '*m')
pl.plot(observer[0],observer[1],'*y',ms=15,mec='k')

Let's create a Gaia survey from this new sample as if we were at the Solar circle

In [ ]:
observer = [8,0,0] #kpc

In [ ]:
name = 'stream'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

In [ ]:
survey = ananke.run()

In [ ]:
survey.length()

In [ ]:
survey_observed = survey[~survey.gaia__dr2_g.isna()]

In [ ]:
survey_observed.length()

In [ ]:
ra = apc.Angle(survey_observed.ra.values*u.deg).wrap_at(180*u.deg)
dec = apc.Angle(survey_observed.dec.values*u.deg).wrap_at(180*u.deg)

pl.hist2d(ra.value, dec.value, bins=[200,100], norm=LogNorm());

pl.xlim(-180,180)
pl.ylim(-90,90)

Here we generated a fraction (1/100th) of the full sample with no magnitude limits, and then selected Gaia observed stars after the fact. Given the results we expect ~40,000 stars from the full survey, which is fairly manageable. Let's now use the inbuilt magnitude limits to generate the full view.

In [ ]:
#limits in either absolute or apparent magnitude or color
cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 21.5,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }
fsample = 1.0

In [ ]:
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

In [ ]:
survey_full = ananke.run()

In [ ]:
survey_full.length()

In [ ]:
survey_observed = survey_full[~survey_full.gaia__dr2_g.isna()]

In [ ]:
survey_observed.length()

we can recover the spawned particles by choosing those with partid == 0:

In [ ]:
survey_parents = survey_full[survey_full.partid==0]

In [ ]:
ra = apc.Angle(survey_observed.ra.values*u.deg).wrap_at(180*u.deg)
dec = apc.Angle(survey_observed.dec.values*u.deg).wrap_at(180*u.deg)

pl.hist2d(ra.value, dec.value, bins=[200,100], norm=LogNorm());

ra_parents = apc.Angle(survey_parents.ra.values*u.deg).wrap_at(180*u.deg)
dec_parents = apc.Angle(survey_parents.dec.values*u.deg).wrap_at(180*u.deg)

pl.plot(ra_parents,dec_parents, '*m', ms=2)

pl.xlim(-180,180)
pl.ylim(-90,90)

Here's the proper motion space (axes are mas/yr)

In [ ]:
pl.hist2d(survey_observed.mul.values,survey_observed.mub.values,bins=np.linspace(-30,30,100),norm=LogNorm());
pl.plot(survey_parents.mul.values,survey_parents.mub.values,'*m', ms=2);
#pl.xlim(-100,100)
#pl.ylim(-100,100)

Now let's look at the same stream observed with LSST, which has an apparent magnitude limit of 26.9 in g and r over 10 years (\~24.5 in one exposure).

In [ ]:
an.display_available_photometric_systems()['padova']['LSST'].mag_names

In [ ]:
cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 26.9,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

#which instrument is surveying ?
photo_sys = 'padova/LSST'


#how are you describing the magnitude/color limits of the survey?
#here we're using LSST r magnitude and g-r color
cmd_magnames = {'magnitude': 'r',
                'color_minuend': 'g',
                'color_subtrahend': 'r'}

setting up the survey will throw a warning because we haven't implemented the LSST error model yet =)

In [ ]:
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

In [ ]:
survey_lsst = ananke.run()

In [ ]:
survey_lsst.length()

That's right, you just made a 4-million-star mock catalog. Do some plotting! go! =)